# WikidWiki Class Description

## attributes
    rootfolder
    datafolder
    pages[]: list of all pages
    pageCount
    
## methods
    __init__(folder_path)
    getAllPageNames () --> returns all pages    getWikiPage (wikipage_name)
    getMatchingPages(functionPageFilter())
    
    createWikiPage (wikipage_name)
    getWikiPageAsString()
    setWikiPageFromString(new_page_as_string)
    getPageSet (Tag_Set)
    getPageSet (Category_Set)
    getPageSet (list_of_page_names)




## WikidWiki Class:

In [1]:
import os,glob
import mmap

from WikidWorker import hasWiki

class WikidWiki(object):
    
    def __init__(self,folder_path):
        """Construct a WikidWiki class using the folder_path as string."""
        # raise error if folder path doesn't exist or contains no wiki:
        if not os.path.exists(folder_path):
            raise Exception("folder " + folder_path + " does not exist!")
        if not hasWiki(folder_path):
            raise Exception("folder " + folder_path + " does not contain a wiki!")

        #set attributes:
        self.rootfolder = folder_path
        self.datafolder = os.path.join(folder_path,'data')
        self.readFileSystem()
        
    def readFileSystem(self):
        """Populates the PageName attributes of WikidWiki class from the filesystem"""
        self.pageNames = self.getAllPageNames()
        self.pageNamesSet = set(self.pageNames)
        self.pageCount = len(self.pageNames)
    
    def __extractWikiFileNamesFromPath(self,wikiFilePath):
        filename = os.path.split(wikiFilePath)[1]
        return os.path.splitext(filename)[0]

    def __getFilePathFromPageName(self,pageName):
        filename = os.path.join(self.datafolder,"%s.wiki" % pageName)
        return filename    
    
    def getAllPageNames(self):
        """Returns a list of PageNames found in data folder."""
        filePathList = glob.glob(self.datafolder +'/*.wiki')
        return [self.__extractWikiFileNamesFromPath(x) for x in filePathList]
    
    def getMatchingPageNamesByFunction(self,matchFunction,matchValue=""):
        """ Passed a match function and a match value to filter pages
            Returns list of pageNames that return true when processed by function matchFunction(matchValue,pagename)
        """
        self.pageNames = self.getAllPageNames()
        if matchValue:
            return [x for x in self.pageNames if matchFunction(matchValue,x)]
        else:
            return [x for x in self.pageNames if matchFunction(x)]            

    def doesPageContainStringLowMem(self,pageName,srchstr):
        filepath = self.__getFilePathFromPageName(pageName)
        
        if os.stat(filepath).st_size > 0:
            f = open(filepath)
            s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
            return s.find(srchstr) != -1
        return False
    
    def doesPageContainString(self,pageName,srchstr):
        filepath = self.__getFilePathFromPageName(pageName)
        return srchstr in open(filepath).read()

    def getPageNamesContainingSearchString(self,srchstr):
        """Returns list of pageNames that contain the srchstr in their content"""
        return [x for x in self.pageNames if self.doesPageContainString(x,srchstr)]  
    
    def getPageNamesContainingSearchStringLowMem(self,srchstr):
        """Returns list of pageNames that contain the srchstr in their content"""
        return [x for x in self.pageNames if self.doesPageContainStringLowMem(x,srchstr)] 

<function hasWiki at 0x104135ed8>PASSED ALL TESTS


In [4]:
# w = WikidWiki('TestWiki')
# w = WikidWiki('BadTestWiki')
# w.getAllWikiPageNames()

#JWiki test:

# wik = WikidWiki("/Users/jaysen/DEV/Projects/WikidLib/NewtWiki")
# print wik.pageCount

jwik = WikidWiki("/Users/jaysen/DEV/Projects/WikidLib/JWiki")
print "pagecount = %i" % jwik.pageCount

p1 = jwik.getPageNamesContainingSearchString("[tag:green")
p2 = jwik.getPageNamesContainingSearchStringLowMem("flag:green")

print "p1(%i)" % len(p1) 
print p1
print
print "p2(%i)" % len(p2) 
print p2

pagecount = 2266
p1(16)
['201211', '20121128', '20130429', '201412', '20150730', 'BrainStorm', 'DeepWaterHorizon', 'FreeFormTyping', 'GreenCognitiveNetworks', 'GreenFlag', 'GreenNetwork', 'InterPersonalProblemResolutions', 'KNitting', 'NicoleTurnerProblemResolutions', 'RefactoringMe', 'WildWikiThoughtStreams']

p2(0)
[]


# Automated Tests:

In [15]:
#TESTS
from nose.tools import assert_equal
from nose.tools import assert_items_equal



class WikidWikiClassTest(object):
    
    def test(self):      
        tc = WikidWiki('TestWiki')
        
        assert_equal(tc.pageNames,['TestOne', 'TestThree', 'TestTwo'])
        assert_equal(tc.pageCount,3)
        
        assert_items_equal(tc.getMatchingPageNamesByFunction(PageNameContainsStringOne),['TestOne'])
        assert_items_equal(tc.getMatchingPageNamesByFunction(PageNameContainsString,'Two'),['TestTwo'])
        assert_items_equal(tc.getMatchingPageNamesByFunction(PageNameContainsString,'Three'),['TestThree'])
        assert_items_equal(tc.getMatchingPageNamesByFunction(PageNameContainsString,'Test'),['TestOne','TestTwo','TestThree'])
        
        print 'class ' + str(tc) + ' - PASSED ALL TESTS'
                     
#run tests
def PageNameContainsStringOne(pageName):
    return 'One' in pageName

def PageNameContainsString(s,pageName):
    return s in pageName

t = WikidWikiClassTest()
t.test()


class <__main__.WikidWiki object at 0x10425ed10> - PASSED ALL TESTS


## PageNameSet Class

In [94]:
class PageNameSet (object):
    
    def __init__(self,pageName):
        
        self.filePath
        self.
        
        
        